## Example of CAMS air quality analysis time series plot from Github CSV repository

CAMS air quality analysis time series data at 50 major cities are available from the [Github](https://github.com/CopernicusAtmosphere/air-quality-covid19-response) repository. You can also retrieve raw fields from the Atmosphere Data Store.

In [ ]:
#!/usr/bin/env python
import pandas as pd
YEAR = 2020
url = f'https://raw.githubusercontent.com/CopernicusAtmosphere/air-quality-covid19-response/master/cams_air_quality_analysis_{YEAR}.csv'
df = pd.read_csv(url,index_col=['basetime', 'city_id'], parse_dates=[0])
print(df.head())

Load metadata and find location ID of a city of interest

In [ ]:
locations_url = 'https://raw.githubusercontent.com/CopernicusAtmosphere/air-quality-covid19-response/master/CAMS_AQ_LOCATIONS_V1.csv'
locations = pd.read_csv(locations_url)
print(locations.head())
city = 'Paris'
locations[locations['name'].str.contains(city)]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors

param = 'NO2'
# df.query("city_id == 'AQ031'").NO2.plot.line(figsize=(15,8))
data = df.query("city_id == 'AQ031'").reset_index()
print(data.head())

fig = plt.figure(figsize=(15,8))
plt.plot(data.basetime, data[param], label=param)
plt.grid()
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.title(f'CAMS European air quality {city} {param} analysis', fontsize=20, pad=20)
# plt.show()

<hr>

Now add climatology information based on CAMS analysis from the past few years

In [ ]:
clim_years = range(2017, YEAR)

past_data = {}

for year in clim_years:
    url = f'https://raw.githubusercontent.com/CopernicusAtmosphere/air-quality-covid19-response/master/cams_air_quality_analysis_{year}.csv'
    df = pd.read_csv(url,index_col=['basetime', 'city_id'], parse_dates=[0])

    data = df.query("city_id == 'AQ031'")[param].droplevel('city_id')
    print(data.head())
    data.index = data.index.map(lambda t: t.replace(year=YEAR))
    
    print(data)
    
    if "min" not in past_data:
        past_data = {
          "min": data,
          "max": data,
          "value": data
        }
    else:
        past_data["min"] =  pd.concat([past_data["min"], data], axis=1).min(axis=1)
        past_data["max"] =  pd.concat([past_data["max"], data], axis=1).max(axis=1)
        past_data["value"] =  pd.concat([past_data["value"], data], axis=1).mean(axis=1)

data_ = past_data

print(data_)

label = "max"
data_['mean_7d'] = data_['value'].rolling(7, center=True, min_periods=5).mean()

fig = plt.figure(figsize=(15,8))
p1 = plt.fill_between(data_["min"].index, data_["min"].values, data_["max"].values, color="#333333", alpha=0.1, label="{}-{} range".format(clim_years[0], clim_years[-1]))
p1 = plt.plot(data_["mean_7d"].index, data_["mean_7d"].values, color="#333333", ls="--", alpha=0.6, label="2017-2019 7d mean", lw=1.5)
